In [2]:
!pip install datasets

     |████████████████████████████████| 298 kB 8.1 MB/s 
     |████████████████████████████████| 243 kB 51.4 MB/s 
     |████████████████████████████████| 1.1 MB 42.5 MB/s 
     |████████████████████████████████| 61 kB 536 kB/s 
     |████████████████████████████████| 132 kB 56.7 MB/s 
     |████████████████████████████████| 271 kB 31.1 MB/s 
     |████████████████████████████████| 160 kB 65.2 MB/s 
     |████████████████████████████████| 192 kB 67.7 MB/s 


In [3]:
from datasets import load_dataset

dataset = load_dataset("imdb")

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [9]:
dataset['train'][0]

{'label': 0,
 'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are f

In [10]:
train= dataset['train']

In [11]:
len(train)

25000

In [12]:
train[0]

{'label': 0,
 'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are f

In [39]:
test= dataset['test']

In [16]:
import pandas as pd
import numpy as np


In [14]:
df_train = pd.DataFrame.from_dict(train)

In [40]:
df_test = pd.DataFrame.from_dict(test)

In [41]:
df_train.head(), df_test.head(), 

(                                                text  ...  compound_score
 0  I rented I AM CURIOUS-YELLOW from my video sto...  ...               0
 1  "I Am Curious: Yellow" is a risible and preten...  ...               0
 2  If only to avoid making this type of film in t...  ...               1
 3  This film was probably inspired by Godard's Ma...  ...               0
 4  Oh, brother...after hearing about this ridicul...  ...               0
 
 [5 rows x 8 columns],
                                                 text  label
 0  I love sci-fi and am willing to put up with a ...      0
 1  Worth the entertainment value of a rental, esp...      0
 2  its a totally average film with a few semi-alr...      0
 3  STAR RATING: ***** Saturday Night **** Friday ...      0
 4  First off let me say, If you haven't enjoyed a...      0)

In [18]:
#Checking for null and blank reviews

df_train.dropna(inplace=True)


blanks = []  # start with an empty list

for i,rv, lb, in df_train.itertuples():  # iterate over the DataFrame
    if type(rv)==str:            # avoid NaN values
        if rv.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
blanks
# df.drop(blanks, inplace=True)

[]

In [20]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [21]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [23]:
df_train['scores'] = df_train['text'].apply(lambda review: sid.polarity_scores(review))

In [25]:
df_train['positive'] = df_train['scores'].apply(lambda score_dict: score_dict['pos'])
df_train['negative'] = df_train['scores'].apply(lambda score_dict: score_dict['neg'])
df_train['neutral'] = df_train['scores'].apply(lambda score_dict: score_dict['neu'])
df_train['compound']  =df_train['scores'].apply(lambda score_dict: score_dict['compound'])

In [26]:
df_train.head()

,text,label,scores,positive,negative,neutral,compound
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"{'neg': 0.056, 'neu': 0.89, 'pos': 0.053, 'com...",0.053,0.056,0.890,-0.2287
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"{'neg': 0.085, 'neu': 0.869, 'pos': 0.046, 'co...",0.046,0.085,0.869,-0.8648
2,If only to avoid making this type of film in t...,0,"{'neg': 0.068, 'neu': 0.798, 'pos': 0.133, 'co...",0.133,0.068,0.798,0.8324
3,This film was probably inspired by Godard's Ma...,0,"{'neg': 0.127, 'neu': 0.749, 'pos': 0.124, 'co...",0.124,0.127,0.749,-0.0924
4,"Oh, brother...after hearing about this ridicul...",0,"{'neg': 0.083, 'neu': 0.876, 'pos': 0.041, 'co...",0.041,0.083,0.876,-0.9358


In [30]:
df_train.label.value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [31]:
# create a list of our conditions
conditions = [
    (df_train['compound'] <= 0.5),
    (df_train['compound'] > 0.5)
    ]
# create a list of the values we want to assign for each condition
values = [0, 1]

# create a new column and use np.select to assign values to it using our lists as arguments
df_train['compound_score'] = np.select(conditions, values)

# display updated DataFrame
df_train.head()

,text,label,scores,positive,negative,neutral,compound,compound_score
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"{'neg': 0.056, 'neu': 0.89, 'pos': 0.053, 'com...",0.053,0.056,0.890,-0.2287,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"{'neg': 0.085, 'neu': 0.869, 'pos': 0.046, 'co...",0.046,0.085,0.869,-0.8648,0
2,If only to avoid making this type of film in t...,0,"{'neg': 0.068, 'neu': 0.798, 'pos': 0.133, 'co...",0.133,0.068,0.798,0.8324,1
3,This film was probably inspired by Godard's Ma...,0,"{'neg': 0.127, 'neu': 0.749, 'pos': 0.124, 'co...",0.124,0.127,0.749,-0.0924,0
4,"Oh, brother...after hearing about this ridicul...",0,"{'neg': 0.083, 'neu': 0.876, 'pos': 0.041, 'co...",0.041,0.083,0.876,-0.9358,0


In [27]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [32]:
accuracy_score(df_train['label'],df_train['compound_score'])

0.71184

In [36]:
print(classification_report(df_train['label'],df_train['compound_score']))

              precision    recall  f1-score   support

           0       0.76      0.61      0.68     12500
           1       0.68      0.81      0.74     12500

    accuracy                           0.71     25000
   macro avg       0.72      0.71      0.71     25000
weighted avg       0.72      0.71      0.71     25000



In [33]:
from matplotlib import pyplot as plt
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import *

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [38]:
#Text classification using Linear Support VEctor Classifier
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC(max_iter=5000))])

In [44]:
y_train = df_train['label']
X_train = df_train['text']
y_test = df_test['label']
X_test = df_test['text']

In [47]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(max_iter=5000))])

In [48]:
predictions = text_clf.predict(X_test)

In [49]:
accuracy_score(y_test,predictions)

0.8772

In [50]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88     12500
           1       0.89      0.87      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



In [51]:
import xgboost

In [52]:
xgboost.XGBClassifier()

XGBClassifier()

In [53]:
#Text classification using Xgboost Classifier
text_clf_xgb = Pipeline([('tfidf', TfidfVectorizer()), ('clf', xgboost.XGBClassifier(max_iter=5000))])

In [54]:
text_clf_xgb.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', XGBClassifier(max_iter=5000))])

In [55]:
predictions = text_clf_xgb.predict(X_test)

In [56]:
accuracy_score(y_test,predictions)

0.81036

In [57]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.85      0.75      0.80     12500
           1       0.78      0.87      0.82     12500

    accuracy                           0.81     25000
   macro avg       0.81      0.81      0.81     25000
weighted avg       0.81      0.81      0.81     25000



In [ ]:
# https://github.com/SMARTA01/Sentiment-Analysis-IMDB

In [ ]:
# https://www.kaggle.com/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews